In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 11:19:05.705945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 11:19:05.810530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 11:19:05.810545: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 11:19:05.831300: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 11:19:06.356064: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 11:19:10.775298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:19:10.775838: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:19:10.775886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:19:10.775926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 11:19:10.775964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 396ms/step - loss: 2.4416 - auc: 0.4529
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.9525 - auc: 0.8224
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0292 - auc: 0.8571
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9857 - auc: 0.8791
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8855 - auc: 0.8881
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7355 - auc: 0.8981
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5480 - auc: 0.9104
Epoch 8/100
1/1 [==============================] - 0s 40ms/step - loss: 1.3987 - auc: 0.9291
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3416 - auc: 0.9284
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2903 - auc: 0.9327
Epoch 11/100
1/1 [==============================] - 0s 38ms/step - loss: 1.2795 

1/1 [==============================] - 0s 26ms/step - loss: 0.1891 - auc: 0.9959
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1914 - auc: 0.9956
Epoch 90/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1864 - auc: 0.9952
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1972 - auc: 0.9946
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1892 - auc: 0.9953
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1909 - auc: 0.9950
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1893 - auc: 0.9951
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1861 - auc: 0.9959
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1834 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1846 - auc: 0.9950
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.0574 - auc: 0.5785
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8158 - auc: 0.8301
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8428 - auc: 0.8595
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7720 - auc: 0.8649
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6162 - auc: 0.8809
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4367 - auc: 0.8949
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3081 - auc: 0.9130
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2588 - auc: 0.9128
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2330 - auc: 0.9061
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1590 - auc: 0.9257
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2019 - auc: 0.9949
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2125 - auc: 0.9935
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2096 - auc: 0.9940
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2069 - auc: 0.9945
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2054 - auc: 0.9945
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2003 - auc: 0.9953
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2044 - auc: 0.9942
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2028 - auc: 0.9944
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2078 - auc: 0.9942
Epoch 97/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2104 - auc: 0.9930
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 2.2163 - auc: 0.5141
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8348 - auc: 0.8254
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9063 - auc: 0.8521
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8413 - auc: 0.8727
Epoch 5/100
1/1 [==============================] - 0s 38ms/step - loss: 1.7048 - auc: 0.8817
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5253 - auc: 0.8975
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3754 - auc: 0.9124
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2924 - auc: 0.9201
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2364 - auc: 0.9247
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1978 - auc: 0.9266
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.1864 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1931 - auc: 0.9943
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1816 - auc: 0.9960
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1917 - auc: 0.9950
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1886 - auc: 0.9947
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1838 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1810 - auc: 0.9961
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1863 - auc: 0.9957
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1808 - auc: 0.9959
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1829 - auc: 0.9958
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.2950 - auc: 0.4816
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7436 - auc: 0.8405
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8247 - auc: 0.8726
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8180 - auc: 0.8841
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7416 - auc: 0.8954
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6154 - auc: 0.9047
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4400 - auc: 0.9167
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3082 - auc: 0.9274
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2088 - auc: 0.9391
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1937 - auc: 0.9298
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1824 - auc: 0.9965
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1863 - auc: 0.9956
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1820 - auc: 0.9968
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1893 - auc: 0.9955
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1844 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1835 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1848 - auc: 0.9959
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1891 - auc: 0.9943
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1883 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1894 - auc: 0.9955
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 346ms/step - loss: 2.1383 - auc: 0.5583
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.9016 - auc: 0.8160
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9091 - auc: 0.8448
Epoch 4/100
1/1 [==============================] - 0s 37ms/step - loss: 1.7687 - auc: 0.8706
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6112 - auc: 0.8810
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4349 - auc: 0.8997
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2933 - auc: 0.9240
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2644 - auc: 0.9162
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2234 - auc: 0.9186
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1372 - auc: 0.9416
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.1828 - auc: 0.9957
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1804 - auc: 0.9963
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1936 - auc: 0.9949
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1789 - auc: 0.9960
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1833 - auc: 0.9956
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1788 - auc: 0.9964
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1847 - auc: 0.9954
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1840 - auc: 0.9954
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1831 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1886 - auc: 0.9947
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.4292 - auc: 0.4099
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8312 - auc: 0.8008
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.9108 - auc: 0.8499
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.9317 - auc: 0.8636
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8760 - auc: 0.8724
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7111 - auc: 0.8915
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5731 - auc: 0.8954
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3893 - auc: 0.9144
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2883 - auc: 0.9203
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2480 - auc: 0.9182
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 22ms/step - loss: 0.2140 - auc: 0.9934
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2168 - auc: 0.9932
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2205 - auc: 0.9922
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2139 - auc: 0.9934
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2211 - auc: 0.9927
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2048 - auc: 0.9947
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2074 - auc: 0.9946
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2091 - auc: 0.9934
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2074 - auc: 0.9941
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2100 - auc: 0.9931
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [9]:
m='remove_MHFP6'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 1.9974 - auc: 0.7820
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8061 - auc: 0.7547
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6477 - auc: 0.8124
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4935 - auc: 0.8738
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3959 - auc: 0.9025
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2846 - auc: 0.9258
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2090 - auc: 0.9357
Epoch 8/100
1/1 [==============================] - 0s 38ms/step - loss: 1.1177 - auc: 0.9532
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0490 - auc: 0.9638
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9912 - auc: 0.9699
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2070 - auc: 0.9923
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2114 - auc: 0.9917
Epoch 90/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2090 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2066 - auc: 0.9922
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1988 - auc: 0.9939
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2061 - auc: 0.9922
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2194 - auc: 0.9897
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2039 - auc: 0.9931
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1996 - auc: 0.9930
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2042 - auc: 0.9921
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [10]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.3905 - auc: 0.4306
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7902 - auc: 0.8183
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8514 - auc: 0.8597
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8875 - auc: 0.8725
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8064 - auc: 0.8830
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6531 - auc: 0.8968
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4766 - auc: 0.9090
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3431 - auc: 0.9180
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2342 - auc: 0.9343
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2266 - auc: 0.9201
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1932 - auc: 0.9954
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2013 - auc: 0.9950
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1975 - auc: 0.9953
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2000 - auc: 0.9950
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2034 - auc: 0.9940
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1978 - auc: 0.9948
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2022 - auc: 0.9942
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1880 - auc: 0.9962
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1996 - auc: 0.9944
Epoch 97/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2000 - auc: 0.9950
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [11]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 525ms/step - loss: 2.1703 - auc: 0.5342
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8471 - auc: 0.8227
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8806 - auc: 0.8508
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8251 - auc: 0.8629
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6532 - auc: 0.8804
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4838 - auc: 0.8917
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 1.3632 - auc: 0.9014
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3056 - auc: 0.8998
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2802 - auc: 0.8928
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1845 - auc: 0.9209
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1976 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1936 - auc: 0.9959
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1946 - auc: 0.9956
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2015 - auc: 0.9946
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2039 - auc: 0.9946
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1961 - auc: 0.9957
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1933 - auc: 0.9959
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1997 - auc: 0.9947
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1977 - auc: 0.9947
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1970 - auc: 0.9952
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [12]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 2.1718 - auc: 0.5376
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8505 - auc: 0.8245
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9158 - auc: 0.8490
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8028 - auc: 0.8754
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6527 - auc: 0.8871
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4544 - auc: 0.9102
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3190 - auc: 0.9257
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2839 - auc: 0.9180
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2388 - auc: 0.9209
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1945 - auc: 0.9254
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 33ms/step - loss: 0.1855 - auc: 0.9960
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1882 - auc: 0.9954
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1841 - auc: 0.9964
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1875 - auc: 0.9950
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1898 - auc: 0.9951
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1836 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1876 - auc: 0.9957
Epoch 95/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1854 - auc: 0.9958
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1847 - auc: 0.9961
Epoch 97/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1954 - auc: 0.9943
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [13]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.9753 - auc: 0.7578
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8168 - auc: 0.7427
Epoch 3/100
1/1 [==============================] - 0s 41ms/step - loss: 1.6700 - auc: 0.8412
Epoch 4/100
1/1 [==============================] - 0s 38ms/step - loss: 1.5558 - auc: 0.8794
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.4271 - auc: 0.9006
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3104 - auc: 0.9197
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2194 - auc: 0.9358
Epoch 8/100
1/1 [==============================] - 0s 51ms/step - loss: 1.1399 - auc: 0.9512
Epoch 9/100
1/1 [==============================] - 0s 39ms/step - loss: 1.0617 - auc: 0.9651
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9863 - auc: 0.9763
Epoch 11/100
1/1 [==============================] - 0s 45ms/step - l

1/1 [==============================] - 0s 35ms/step - loss: 0.1905 - auc: 0.9949
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1881 - auc: 0.9959
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1949 - auc: 0.9942
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1820 - auc: 0.9963
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1832 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1815 - auc: 0.9960
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1861 - auc: 0.9958
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1840 - auc: 0.9960
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1854 - auc: 0.9956
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1859 - auc: 0.9955
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [14]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('10feature_remove_one2.csv',index=False)
print(de)

                  model       auc
0             all_model  0.945163
1     remove_AtomPairFP  0.930114
2   remove_Connectivity  0.945813
3         remove_Charge  0.943510
4          remove_Kappa  0.945217
5           remove_MAP4  0.941880
6          remove_MHFP6  0.947293
7            remove_MOE  0.943533
8       remove_MorganFP  0.943590
9       remove_Topology  0.944824
10     remove_TorsionFP  0.946800
